# Load Libraries

In [1]:
import os
import warnings
import logging
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

# Set up logging
logging.basicConfig(level=logging.INFO)

# Suppress warnings
warnings.filterwarnings("ignore")

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [2]:
# Define file paths using environment variables
DATA_FOLDER_PATH = os.getenv('DATA_FOLDER_PATH')
DATA_NAME = os.getenv('DATA_NAME')
GEOINFO_NAME = os.getenv('GEOINFO_NAME')

if not DATA_FOLDER_PATH or not DATA_NAME or not GEOINFO_NAME:
    logging.error("Environment variables DATA_FOLDER_PATH, DATA_NAME, or GEOINFO_NAME are not set.")
    sys.exit(1)

# Construct full file paths
DATA_PATH = os.path.join(DATA_FOLDER_PATH, DATA_NAME + '.parquet')
GEOINFO_PATH = os.path.join(DATA_FOLDER_PATH, GEOINFO_NAME + '.geojson')

# Load datasets
try:
    # Read Parquet file with pandas
    data = pd.read_parquet(DATA_PATH)
    logging.info(f"Data loaded successfully from {DATA_PATH}")
except Exception as e:
    logging.error(f"Error loading data from {DATA_PATH}: {e}")
    sys.exit(1)

try:
    # Read GeoJSON file with geopandas    
    geoinfo = gpd.read_file(GEOINFO_PATH)
    logging.info(f"Geoinfo loaded successfully from {GEOINFO_PATH}")
except Exception as e:
    logging.error(f"Error loading geoinfo from {GEOINFO_PATH}: {e}")
    sys.exit(1)

INFO:root:Data loaded successfully from C:/Users/Pooya/Dropbox/IRIMO/Export/Iran_Data_1950_2025\IRIMO_Data.parquet
INFO:root:Geoinfo loaded successfully from C:/Users/Pooya/Dropbox/IRIMO/Export/Iran_Data_1950_2025\IRIMO_GeoInfo.geojson


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4928062 entries, 0 to 4928061
Data columns (total 55 columns):
 #   Column             Dtype         
---  ------             -----         
 0   station_id         category      
 1   station_name       category      
 2   region_id          category      
 3   region_name        category      
 4   lat                float64       
 5   lon                float64       
 6   station_elevation  float64       
 7   date               datetime64[ns]
 8   ff_max             float64       
 9   dd_max             float64       
 10  ffm                float64       
 11  tmax               float64       
 12  tmin               float64       
 13  tm                 float64       
 14  pm                 float64       
 15  p0m                float64       
 16  ewm                float64       
 17  rrr24              float64       
 18  nm                 float64       
 19  nmax               float64       
 20  essmax             float

In [4]:
data.region_name.unique().tolist()

['Airforce',
 'Alborz',
 'Ardebil',
 'Azarbayjan-E-Gharbi',
 'Azarbayjan-E-Sharghi',
 'Bushehr',
 'Chaharmahal Va Bakhtiari',
 'Esfahan',
 'Fars',
 'Gilan',
 'Golestan',
 'Hamedan',
 'Hormozgan',
 'Ilam',
 'Kerman',
 'Kermanshah',
 'Khohgiluyeh Va Boyerahmad',
 'Khorasan Razavi',
 'Khuzestan',
 'Kordestan',
 'Lorestan',
 'Markazi',
 'Mazandaran',
 'North Khorasan',
 'Qazvin',
 'Qom',
 'Semnan',
 'Sistan Va Baluchestan',
 'South Khorasan',
 'Tehran',
 'Yazd',
 'Zanjan']

# Filter Data

In [6]:
selected_regions = ["Khorasan Razavi", "South Khorasan", "North Khorasan"]

selected_columns = [
    'station_id',
    'station_name',
    'region_id',
    'region_name',
    'lat',
    'lon',
    'station_elevation',
    'date',
    'tmax',
    'tmin',
    'tm',
    'umax',
    'umin',
    'um',
    'ffm',
    'sshn',
    'rrr24',
]

df = data.filter(items=selected_columns).query("region_name in @selected_regions").copy()

def remove_unused_if_cat(s):
    if pd.api.types.is_categorical_dtype(s.dtype):
        return s.cat.remove_unused_categories()
    else:
        return s

df = df.apply(remove_unused_if_cat)

# df.dropna(subset=selected_columns, inplace=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531359 entries, 2633701 to 4538369
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   station_id         531359 non-null  category      
 1   station_name       531359 non-null  category      
 2   region_id          531359 non-null  category      
 3   region_name        531359 non-null  category      
 4   lat                531359 non-null  float64       
 5   lon                531359 non-null  float64       
 6   station_elevation  531359 non-null  float64       
 7   date               531359 non-null  datetime64[ns]
 8   tmax               522466 non-null  float64       
 9   tmin               511502 non-null  float64       
 10  tm                 512414 non-null  float64       
 11  umax               394472 non-null  float64       
 12  umin               394447 non-null  float64       
 13  um                 394472 non-null  float6

# Filter Data

In [9]:
# df.query('date >= "1983-01-01" and date <= "2022-12-31"', inplace=True)
df.reset_index(drop=True, inplace=True)

In [10]:
# Calculate number of records per station`
station_record_counts = df['station_id'].value_counts().reset_index()

station_record_counts['year'] = station_record_counts["count"] // 365

station_record_counts


,station_id,count,year
0,40745,27272,74
1,40743,25902,70
2,40809,25628,70
3,40762,24294,66
4,40791,21758,59
...,...,...,...
78,19062,243,0
79,18606,235,0
80,18604,228,0
81,51202,101,0


# Export Data

In [12]:
# To pickle in data folder
with open('../../data/Khorasan_Data_1950_2025.pkl', 'wb') as f:
    pickle.dump(df, f)